# Оптимизация маршрута при помощи OpenRouteService и Ortools

Представим ситуацию: Мы - туристы, увлекающиеся изобразительным искусстовом, которые приехали в Москву на пару дней, и мы хотим успеть осмотреть как можно больше арт-объектов в центре города. Перед нами стоит задача - оптимизировать маршрут, чтобы обойти все места как можно быстрее.

In [1]:
import folium
from shapely import wkt, geometry


На сайте [Openrouteservice](https://openrouteservice.org) найдем API ключ и с помощью сервиса [Wicket](https://arthur-e.github.io/Wicket/sandbox-gmaps3.html) выделяем область в которой будем искать места интереса

In [2]:
api_key ='5b3ce3597851110001cf62482fc47888b6bb47d9ac5e20bbfa7dff7c'
wkt_string = 'POLYGON((37.61194026772619 55.74738114319582,37.60945117776037 55.7493134624298,37.601554754420526 55.75095585853057,37.60069644753576 55.754820047193846,37.59846484963537 55.758104306587434,37.603614690943964 55.76341649361732,37.60893619362951 55.76612060176926,37.61339938943029 55.76814855981075,37.620094183131464 55.76737601680498,37.628333929225214 55.76670002911673,37.63485706154943 55.76631374517526,37.63846195046545 55.76525144460154,37.64258182351232 55.762933597426496,37.64532840554357 55.75955315663614,37.646530035182245 55.75771793748363,37.648246648951776 55.75588263196736,37.64721668069006 55.75259818548602,37.64532840554357 55.75105246791257,37.642066839381464 55.750086363325885,37.63949191872717 55.74747776143056,37.63211047951818 55.7493134624298,37.62541568581701 55.74950668908941,37.62026584450842 55.74902362064598,37.61494434182287 55.7481540823742,37.61194026772619 55.74738114319582))'
# Выделяем бульварное кольцо
aoi_geom = wkt.loads(wkt_string)
aoi_coords = list(aoi_geom.exterior.coords)# Список координат области в которой будет происходить поиск
aoi_coords = [(y, x) for x, y in aoi_coords]# Для корректного отображения на карте необходимо поменять широту и долготу местами
aoi_centroid = aoi_geom.centroid

Создаем карту и выделяем выбранную нами область

In [3]:
m = folium.Map(tiles='Stamen Toner', location=(aoi_centroid.y, aoi_centroid.x), zoom_start=13)
folium.vector_layers.Polygon(aoi_coords,
                             color='#ffd699',
                             fill_color='#ffd699',
                             fill_opacity=0.2,
                             weight=3).add_to(m)
m

In [4]:
from openrouteservice import client, places

ors = client.Client(key=api_key)

Воспользуемся [**Places API**](https://openrouteservice.org/dev/#/api-docs/pois) и отправлим запрос на поиск интересующих нас мест. В этом случае будем искать картинные галереи и арт-площадки они находятся в разделе `arts_and_culture : 130` под номерами [132] и [131] соответственно. Больше вариантов для поиска можно найти [здесь](https://giscience.github.io/openrouteservice/documentation/Places.html)
>Область ограничена в размерах

In [5]:
aoi_json = geometry.mapping(geometry.shape(aoi_geom)) # Переводим информацию из wkt в Geojson для обработки ее в ORS
query = {'request': 'pois',
         'geojson': aoi_json,
         'filter_category_ids': [131],
         'sortby': 'distance'}
art_spaces = ors.places(**query)['features']  # Выполняем запрос и получаем интересующую нас информацию
query = {'request': 'pois',
         'geojson': aoi_json,
         'filter_category_ids': [132],
         'sortby': 'distance'}
galleries = ors.places(**query)['features']
art_places = art_spaces+galleries

Посчитаем колчество арт-объектов, а затем добавим их на карту

In [6]:
print("\nКоличество : {}".format(len(art_places)))


Количество : 20


Добавим найденные арт-объекты на карту

In [7]:
art_addresses = []
for feat in art_places:
    lon, lat = feat['geometry']['coordinates']
    name = ors.pelias_reverse(point=(lon, lat))['features'][0]['properties']['name']
    popup = "<strong>{0}</strong><br>Lat: {1:.3f}<br>Long: {2:.3f}".format(name, lat, lon) # всплывающий текст
    icon = folium.map.Icon(color='lightgray',
                           icon_color='#b5231a',
                           icon='paint-brush',  # Можно использовать любые иконки fontawesome v4
                           prefix='fa')
    folium.map.Marker([lat, lon], icon=icon, popup=popup).add_to(m)
    art_addresses.append(name)

m

Места интереса найдены, теперь необходимо получить информацию об их взаимном расположении, для этого получим матрицу расстояний, используя [**Matrix API**](https://openrouteservice.org/dev/#/api-docs/matrix).
Так как время ограничено, то передвигаться будем на транспорте.
>Матрица должна содержать менее 3500 путей, иначе говоря должна быть иметь размер меньше чем 59x59

In [8]:
from openrouteservice import distance_matrix

art_coords = [feat['geometry']['coordinates'] for feat in art_places]

request = {'locations': art_coords,
           'profile': 'driving-car',
           'metrics': ['duration']} # в матрице будем хранить время в пути между двумя точками

art_matrix = ors.distance_matrix(**request)
print("Посчитано {}x{} путей.".format(len(art_matrix['durations']), len(art_matrix['durations'][0])))


Посчитано 20x20 путей.


Теперь перед нами стоит классическая задача коммивояжера, заключающаяся в поиске самого выгодного (в данном случае самого быстрого) маршрута проходящего через все точки интереса и возвращающегося в начальную точку. Для решения этой проблемы воспользуемся [**ortools**](https://github.com/google/or-tools)

In [9]:
from ortools.constraint_solver import pywrapcp

tsp_size = len(art_addresses) # количество точек
num_routes = 1 # количество путей
start = 0  # начальная точка


optimal_coords = []

if tsp_size > 0:
    manager = pywrapcp.RoutingIndexManager(tsp_size, num_routes, start)
    routing = pywrapcp.RoutingModel(manager)


    # Создадим функцию возвращающую продолжительность пути из точки с индексом from в точку с индексом to
    def distance_callback(from_index, to_index):
        # Преобразуем переменную используемую при построении маршрута в индексы матрицы расстояний
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return int(art_matrix['durations'][from_node][to_node])


    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index) # устанавливаем цену перемещения
    # Решаем задачу поиска маршрута
    assignment = routing.Solve()
    if assignment:
        # Полное время маршрута
        print("Время в пути: " + str(round(assignment.ObjectiveValue(), 3) // 60) + " минута\n")
        index = routing.Start(start)  # Индекс начальной точки
        route = ''
        for node in range(routing.nodes()): # Создаем Маршрут
            optimal_coords.append(art_coords[manager.IndexToNode(index)]) # Сохраняем координаты точек, для визуализации маршрута
            route += str(art_addresses[manager.IndexToNode(index)]) + ' -> '
            index = assignment.Value(routing.NextVar(index))
        route += str(art_addresses[manager.IndexToNode(index)])
        optimal_coords.append(art_coords[manager.IndexToNode(index)])
        print("Маршрут:\n" + route)



Время в пути: 41 минута

Маршрут:
7 с2 Кисельный тупик -> Nadja Brykina Gallery -> Vincent -> КЦ Хитровка -> Граунд Солянка -> Выставочный зал Московского союза художников -> Art & Brut -> Фонд культуры "Екатерина -> Центральный клуб МВД -> Новое пространство Театра наций -> ДК Петлюра -> Пунктум -> MustART -> Brusov Art Space -> Московский дом композиторов -> Cube.Moscow -> Шаурмен -> 3/8 с5 улица Ильинка -> Центральный дом работников искусств -> Эритаж -> 7 с2 Кисельный тупик


Маршрут найден осталось только проложить его на карте. И для наглядности проложим более-менее случайный маршрут по этим точкам, в порядке их Geojson данных

In [10]:
def style_function(color): # Стилизация линий маршрута
    return lambda feature: dict(color=color,
                                weight=3,
                                opacity=1)


# Случайный Маршрут
art_coords.append(art_coords[0])
request = {'coordinates': art_coords,
           'profile': 'driving-car',
           'geometry': 'true',
           'format_out': 'geojson',
           }
random_route = ors.directions(**request)

folium.features.GeoJson(data=random_route,
                        name='Random Bar Crawl',
                        style_function=style_function('#ff0000'), # красный цвет
                        overlay=True).add_to(m)
m

In [11]:
# Оптимальный маршрут
request['coordinates'] = optimal_coords
optimal_route = ors.directions(**request)
folium.features.GeoJson(data=optimal_route,
                        name='Optimal Bar Crawl',
                        style_function=style_function('#6666ff'),
                        overlay=True).add_to(m)

m.add_child(folium.map.LayerControl())
m

Несложно заметить, что фиолетовый маршрут выглядит более привлекательно, по сравнению с красным

И в заключение посмотрим сколько времени мы сэкономили оптимизируя маршрут

In [14]:
optimal_duration = 0
random_duration = 0

optimal_duration = optimal_route['features'][0]['properties']['summary']['duration'] // 60
random_duration = random_route['features'][0]['properties']['summary']['duration'] // 60

print("Длительность оптимального маршрута: {0:.0f} минута\nДлительность случайного пути: {1:.0f} минут".format(optimal_duration,
                                                                                         random_duration))

Длительность оптимального маршрута: 41 минута
Длительность случайного пути: 96 минут


Экономия составила 55 минут,и например, если бы мы решили в качестве транспорта использовать такси, то по стандартному тарифу агрегатора, мы бы сэкономили около 500 рублей. 